In [13]:
import json

from typing import List, Tuple

from cjw.aistory.utilities.GptPortal import GptPortal
import os


In [14]:
key = os.environ.get("OPENAI_KEY")
os.environ["OPENAI_API_KEY"] = key
gpt = GptPortal.of(key)

In [15]:
specs = []
conversation = []
code = ""

In [16]:
def addMessage(content: str | List[str], role: str = "user"):
    global conversation
    if isinstance(content, str):
        content = [content]

    conversation.append({
        "role": role,
        "content": '\n'.join(content)
    })

In [17]:
def addSpecs(content: str | List[str]):
    global specs
    if isinstance(content, str):
        content = [content]

    specs += content

In [27]:
def makeMessages():
    global conversation, specs

    return (
            [{
                "role": "system",
                "content": '\n\n'.join(specs)
            }] +
            conversation +
            [{
                "role": "user",
                "content": code
            }]
    )


In [43]:
addSpecs([
    "We are to write a python class 'ClassGen' together.  "
    "It will take a set of specifications, written in natural languages, and generate a python class definition.",
    "Generate the complete ClassGen each time."
    "**Specifications**",
])

In [20]:
addSpecs([
    "### Upon creating a ClassGen object, a user shall be able to state the purpose of the generated class.",
    "### ClassGen shall allow a user to add specifications to the generated class.",
    "### A user shall be able to call the generate() method, which output the class definition of the specs using GPT-4.",
    "### A user shall be able give detailed instructions to fine-tune and correct the generated code.",
    "### The response form GPT-4 may contain code descriptions.  ClassGen separate code from the description before returning them.  "
    "The code starts with a line of '```python' and ends with a line of '```'.",
    "### The generate() method shall use the specifications, the detail instructions, and the latest version generated code to generate the next version of teh code",
])

In [21]:
addSpecs(
    "### ClassGen shall use an external module GptPortal class to access GPT-4. "
    "An example of it's usage:\n"
    "```python"
    "from cjw.aistory.utilities.GptPortal import GptPortal\n"
    "key = os.environ.get('OPENAI_API_KEY')\n"
    "gpt = GptPortal.of(key)\n"
    "response = await gpt.chatCompletion(messages)\n"
    "```\n"
    "Where the 'messages' is the same as OpenAI GPT-4 ChatCompletion messages structure, the response['content'] contains the response, and response['role'] is 'assistant'.",
)


In [22]:
addSpecs([
    "### ClassGen shall provide a rewrite_specs() method, which update and consolidate the specs bsed on the instructions."
])

In [28]:
def extract_code(content: str) -> Tuple[str, str]:
    start_marker = '```python\n'
    if start_marker in content:
        code_start = content.find(start_marker) + len(start_marker)
        code_end = content.index("```\n", code_start)
        python = content[code_start:code_end].strip()
        description = content[:code_start] + '[old code omitted]\n' + content[code_end:]
        return python, description
    else:
        return '', content


In [24]:
async def generate():
    messages = makeMessages()
    response = await gpt.chatCompletion(messages)
    python, description = extract_code(response["content"])
    addMessage(description, role="assistant")
    return python


In [25]:
code = await generate()
print(code)

import os
from cjw.aistory.utilities.GptPortal import GptPortal

class ClassGen:
    def __init__(self, purpose):
        self.specs = []
        self.detail_instructions = []
        self.latest_code = ""
        self.purpose = purpose
        self.key = os.environ.get('OPENAI_API_KEY')
        
    def add_specification(self, spec: str, detail: str = ""):
        self.specs.append(spec)
        if detail:
            self.detail_instructions.append(detail)
        
    def rewrite_specs(self, instructions):
        self.specs.clear()
        self.specs.extend(instructions)
        
    def generate(self):
        gpt = GptPortal.of(self.key)
        messages = self.get_messages()
        response = await gpt.chatCompletion(messages)
        return self.separate_code_from_description(response['content'])
        
    def get_messages(self):
        return [
            {"role": "system", "content": self.purpose},
            *[
                {"role": "user", "content": spec}
       

In [44]:
addMessage([
    "1. generate() do not overwrite latest_code.  Return it so the user has a chance to manually change it.",
    "2. generate() shall insert '[old code omitted]' to the description, and add it to the messages to GPT-4 as 'assistant'.",
    "3. rewrite_specs() shall also use GptPortal to rewrite the specs based on additional inputs from the user's instructions."
])

In [34]:
code = await generate()
print(code)

import os
from cjw.aistory.utilities.GptPortal import GptPortal

class ClassGen:
    def __init__(self, purpose):
        self.specs = []
        self.detail_instructions = []
        self.latest_code = ""
        self.purpose = purpose
        self.key = os.environ.get('OPENAI_API_KEY')
        
    def add_specification(self, spec: str, detail: str = ""):
        self.specs.append(spec)
        if detail:
            self.detail_instructions.append(detail)
        
    async def rewrite_specs(self, instructions):
        self.specs.clear()
        self.specs.extend(instructions)
        gpt = GptPortal.of(self.key)
        messages = [
            {"role": "system", "content": "Rewrite the following specifications."},
            *[
                {"role": "user", "content": instruction}
                for instruction in instructions
            ],
        ]
        response = await gpt.chatCompletion(messages)
        return response['content']
        
    async def generate(self

In [40]:
# code = """
# import os
# from cjw.aistory.utilities.GptPortal import GptPortal
#
# class ClassGen:
#     def __init__(self, purpose):
#         self.specs = []
#         self.detail_instructions = []
#         self.latest_code = ""
#         self.purpose = purpose
#         self.key = os.environ.get('OPENAI_API_KEY')
#
#     def add_specification(self, spec: str, detail: str = ""):
#         self.specs.append(spec)
#         if detail:
#             self.detail_instructions.append(detail)
#
#     async def rewrite_specs(self, instructions):
#         self.specs.clear()
#         self.specs.extend(instructions)
#         gpt = GptPortal.of(self.key)
#         messages = [
#             {"role": "system", "content": "Rewrite the following specifications."},
#             *[
#                 {"role": "user", "content": instruction}
#                 for instruction in instructions
#             ],
#         ]
#         response = await gpt.chatCompletion(messages)
#         return response['content']
#
#     async def generate(self):
#         gpt = GptPortal.of(self.key)
#         messages = self.get_messages()
#         description, latest_code, = "", ""
#         if self.latest_code:
#             description = "[old code omitted]"
#             latest_code = self.latest_code
#         messages.append({"role": "assistant", "content": description})
#         response = await gpt.chatCompletion(messages)
#         new_description, new_code = self.separate_code_from_description(response['content'])
#         return new_description, new_code
#
#     def get_messages(self):
#         return [
#             {"role": "system", "content": self.purpose},
#             *[
#                 {"role": "user", "content": spec}
#                 for spec in self.specs
#             ],
#             *[
#                 {"role": "user", "content": instruction}
#                 for instruction in self.detail_instructions
#             ],
#             {"role": "assistant", "content": self.latest_code}
#         ]
#
#     def separate_code_from_description(self, content):
#         content_lines = content.split('\n')
#         code_start_index = content_lines.index('```python')
#         code_end_index = content_lines.index('```')
#
#         code = "\n".join(content_lines[code_start_index + 1: code_end_index])
#
#         description = "\n".join(content_lines[:code_start_index] + content_lines[code_end_index + 1:])
#         return description, code
# """

In [47]:
addMessage([
    "1. rewrite_specs() does not modify self.specs.  Return the result so the user has a chance to modify them.",
    "2. rewrite_specs() shall include both the old specs and the instruction for rewriting.",
    "3. You misunderstood the last instruction about '[old code omitted]'.  I meant you to add it to separate_code_from_description() where you extract the code from the response of GPT-4.  Then, in generate() add the description to the message as assistant's."
])


In [49]:
code = await generate()
print(code)

import os
from cjw.aistory.utilities.GptPortal import GptPortal

class ClassGen:
    def __init__(self, purpose):
        self.specs = []
        self.detail_instructions = []
        self.latest_code = ""
        self.purpose = purpose
        self.key = os.environ.get('OPENAI_API_KEY')
        
    def add_specification(self, spec: str, detail: str = ""):
        self.specs.append(spec)
        if detail:
            self.detail_instructions.append(detail)
        
    async def rewrite_specs(self, instructions: list):
        gpt = GptPortal.of(self.key)
        messages = [
            {"role": "system", "content": "You are to rewrite the following specifications."},
            {"role": "user", "content": "\n".join(self.specs + instructions)}
        ]
        response = await gpt.chatCompletion(messages)
        new_specs = response['choices'][0]['message']['content']
        return new_specs
        
    async def generate(self):
        gpt = GptPortal.of(self.key)
        mess

In [51]:
addMessage([
    "1. You can instantiate GptPortal at the constructor, so you cna use self.gpt in both generate() and rewrite_specs().",
    "2. generate() only return the code.  The description shall be kept in the message so the conversation between the user and the assistant cab be provided in the get_messages() method.",
    "3. in rewrite_specs(), the response already has the new specs in response['content'].",
    "4. You need to tell GPT-4 to modify and consolidate the specs, with the additional input from the instruction.  Some of the instructions are to correct the mistakes form GPT-4, so shouldn't be included in the specs."
])

In [54]:
# code = """
# import os
# from cjw.aistory.utilities.GptPortal import GptPortal
#
# class ClassGen:
#     def __init__(self, purpose):
#         self.specs = []
#         self.detail_instructions = []
#         self.latest_code = ""
#         self.purpose = purpose
#         self.key = os.environ.get('OPENAI_API_KEY')
#
#     def add_specification(self, spec: str, detail: str = ""):
#         self.specs.append(spec)
#         if detail:
#             self.detail_instructions.append(detail)
#
#     async def rewrite_specs(self, instructions: list):
#         gpt = GptPortal.of(self.key)
#         messages = [
#             {"role": "system", "content": "You are to rewrite the following specifications."},
#             {"role": "user", "content": "\n".join(self.specs + instructions)}
#         ]
#         response = await gpt.chatCompletion(messages)
#         new_specs = response['choices'][0]['message']['content']
#         return new_specs
#
#     async def generate(self):
#         gpt = GptPortal.of(self.key)
#         messages = self.get_messages()
#         response = await gpt.chatCompletion(messages)
#         description, new_code = self.separate_code_from_description(response['content'])
#         messages.append({"role": "assistant", "content": description})
#         return description, new_code
#
#     def get_messages(self):
#         return [
#             {"role": "system", "content": self.purpose},
#             *[
#                 {"role": "user", "content": spec}
#                 for spec in self.specs
#             ],
#             *[
#                 {"role": "user", "content": instruction}
#                 for instruction in self.detail_instructions
#             ],
#             {"role": "assistant", "content": self.latest_code}
#         ]
#
#     def separate_code_from_description(self, content):
#         content_lines = content.split('\n')
#         code_start_index = content_lines.index('```python')
#         code_end_index = content_lines.index('```')
#
#         code = "\n".join(content_lines[code_start_index + 1: code_end_index])
#         description = "\n".join(content_lines[:code_start_index] + ["[old code omitted]"] + content_lines[code_end_index+1:])
#         return description, code
# """
#
# conversation = conversation[:-1]

In [55]:
code = await generate()
print(code)

from cjw.aistory.utilities.GptPortal import GptPortal
import os

class ClassGen:
    def __init__(self, purpose):
        self.purpose = purpose
        self.specs = []
        self.detail_instructions = []
        self.latest_code = ""
        self.gpt = GptPortal.of(os.environ.get('OPENAI_API_KEY'))

    def add_specification(self, spec: str, detail: str = ""):
        self.specs.append(spec)
        if detail:
            self.detail_instructions.append(detail)

    async def generate(self):
        messages = self.get_messages()
        response = await self.gpt.chatCompletion(messages)
        description, new_code = self.separate_code_from_description(response['choices'][0]['message']['content'])
        self.latest_code = new_code
        messages.append({"role": "assistant", "content": description})
        return new_code

    def get_messages(self):
        return [
            {"role": "system", "content": self.purpose},
            *[
                {"role": "user", "conte

In [58]:
# code = """
# from cjw.aistory.utilities.GptPortal import GptPortal
# import os
#
# class ClassGen:
#     def __init__(self, purpose):
#         self.purpose = purpose
#         self.specs = []
#         self.detail_instructions = []
#         self.latest_code = ""
#         self.gpt = GptPortal.of(os.environ.get('OPENAI_API_KEY'))
#
#     def add_specification(self, spec: str, detail: str = ""):
#         self.specs.append(spec)
#         if detail:
#             self.detail_instructions.append(detail)
#
#     async def generate(self):
#         messages = self.get_messages()
#         response = await self.gpt.chatCompletion(messages)
#         description, new_code = self.separate_code_from_description(response['choices'][0]['message']['content'])
#         self.latest_code = new_code
#         messages.append({"role": "assistant", "content": description})
#         return new_code
#
#     def get_messages(self):
#         return [
#             {"role": "system", "content": self.purpose},
#             *[
#                 {"role": "user", "content": spec}
#                 for spec in self.specs
#             ],
#             *[
#                 {"role": "user", "content": instructions}
#                 for instructions in self.detail_instructions
#             ],
#             {"role": "assistant", "content": self.latest_code}
#         ]
#
#     async def rewrite_specs(self, instructions: list):
#         messages = [
#             {"role": "system", "content": 'You are a helpful assistant.'},
#             {"role": "user", "content": f'Help me to rewrite and consolidate the following specifications based on the additional instructions: {" ".join(self.specs + instructions)}'}
#         ]
#         response = await self.gpt.chatCompletion(messages)
#         return response['choices'][0]['message']['content']
#
#     def separate_code_from_description(self, content):
#         content_lines = content.split('\n')
#         code_start_index = content_lines.index('```python')
#         code_end_index = content_lines.index('```')
#         code = "\n".join(content_lines[code_start_index + 1: code_end_index])
#         description = "\n".join(content_lines[:code_start_index] + ["[old code omitted]"] + content_lines[code_end_index+1:])
#         return description, code
#
# """
#
# conversation = conversation[:-1]

In [60]:
addSpecs([
    "1. Again, do not change self.latest_code.  Just return the new code.",
    "2. In both generate and rewriet_specs, replace response['choices'][0]['message']['content'] with response['content'].",
    "3. If you just mix specs and instruction together in rewrite_specs(), how can GPT-4 know what to rewrite?  How can GPT-4 know what were corrections of mistakes that should not be used to change the specs?",
    "ALWAYS GENERATE COMPLETE CLASS!!"
])


In [61]:
code = await generate()
print(code)

import os
from cjw.aistory.utilities.GptPortal import GptPortal

class ClassGen:
    def __init__(self, purpose: str):
        self.purpose = purpose
        self.specs = []
        self.detail_instructions = []
        self.latest_code = ''
        self.gpt = GptPortal.of(os.environ.get('OPENAI_API_KEY'))
        self.messages = [{"role": "system", "content": self.purpose}]

    def add_specification(self, spec: str, detail: str = ""):
        self.specs.append(spec)
        self.messages.append({"role": "user", "content": spec})
        if detail:
            self.detail_instructions.append(detail)
            self.messages.append({"role": "user", "content": detail})

    async def generate(self):
        if self.latest_code:
            self.messages.append({"role": "assistant", "content": self.latest_code})        
        response = await self.gpt.chatCompletion(self.messages)
        description, code = self.separate_code_from_description(response['content'])
        self.message

In [62]:
exec(code)

In [68]:
gen1 = ClassGen(
    "We are to write a python class 'ClassGen' together.  "
    "It will take a set of specifications, written in natural languages, and generate a python class definition."
    "Generate the complete ClassGen each time."
)

for s in specs:
    gen1.add_specification(s)

gen1.latest_code = code

gen1code = await gen1.generate()
print(gen1code)

In [69]:
gen1.

import os
from cjw.aistory.utilities.GptPortal import GptPortal

class ClassGen:
    def __init__(self, purpose: str):
        self.purpose = purpose
        self.specs = []
        self.detail_instructions = []
        self.latest_code = ''
        self.gpt = GptPortal.of(os.environ.get('OPENAI_API_KEY'))
        self.messages = [{"role": "system", "content": self.purpose}]

    def add_specification(self, spec: str, detail: str = ""):
        self.specs.append(spec)
        self.messages.append({"role": "user", "content": spec})
        if detail:
            self.detail_instructions.append(detail)
            self.messages.append({"role": "user", "content": detail})

    async def generate(self):
        if self.latest_code:
            self.messages.append({"role": "assistant", "content": self.latest_code})        
        response = await self.gpt.chatCompletion(self.messages)
        description, code = self.separate_code_from_description(response['content'])
        self.message